# Exploration of Database flow of data

Here we wish to explore the way our data flows through the database. We are interested in figuring out where all the journeys from the *Journeys* table are lost when trying to connect to the table *SJWaypoints*. The SQL query we wish to analysise is the following: 

```SQL
SELECT J.Id as JId, J.CreatedOn, J.SearchStart, J.SearchEnd, J.StartStop, J.EndStop,
    J.StartZone, J.EndZone, J.internalStartZones, J.internalValidZones, 
    SJWaypoints._id as StopId, SJWaypoints.Id,  SJWaypoints.Name, SJWaypoints.Latitude, 
    SJWaypoints.Longitude, SJWaypoints.[Type], SJWaypoints.Line, SJWaypoints.SJSearchJourney_Id as SJId
FROM Journeys J
    JOIN Tickets ON J.Id = Tickets.Journey_Id
    JOIN Orders ON Orders.Id = Tickets.OrderId
    JOIN SJSearchJourneys SJ ON SJ.Id = Orders.JourneyClasses_Id
    JOIN SJWaypoints ON SJWaypoints.SJSearchJourney_Id = SJ.Id
WHERE J.CreatedOn BETWEEN '2022-12-01 00:00:00' and '2023-01-01 00:00:00'
```

We will be splitting up the query into parts exploring the amount of data and the relationship between the two joined tables.

From our starting points, we run the query:
```SQL
SELECT COUNT(*)
FROM Journeys
WHERE Journeys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```

and see that the month has 941,093 registered journeys. How does that number change throughout our joinings?


## Journeys and Tickets

*Journeys* is our starting point for our data since this table consist of actually traveled (bought) journeys. From our ER-Diagram, we notice that the *Journeys* and *Tickets* tables have a 1 - 1 relationship through *Journeys*'s primary key **Id** and *Tickets*'s foreign key **Journey_Id**.

What we expect to see from this join, is the amount of results to be the same.  

The SQL Query in use is the following:
``` SQL
SELECT COUNT(*)
FROM Journeys
    JOIN Tickets ON Journeys.Id = Tickets.Journey_Id
WHERE Journeys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
--- ANSWER: **870,115**

The *Ticket* table does not necessarily contain explict relevant information for our purpose. If so, the column **Type** might be an indicator for whether a journey is from a standard ticket or a 'pendler-kort', which we are not interested in keeping in our training data. 


## Tickets and Orders

Here we wish to look at the relationship between *Tickets* and *Orders*. We expect each ticket to match with an entry in orders. We expect this relationship to be 1 to many, such that the expected count(*) is **LESS** than the 'original' 941,093 results. 

```SQL
SELECT COUNT(*)
FROM Tickets
    JOIN Orders ON Tickets.OrderId = Orders.Id
WHERE Tickets.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```

We also run the following:
```SQL
SELECT COUNT(*)
FROM Tickets
WHERE Tickets.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```


## Orders and SJSearchJourneys

We expect to see a large drop in number in this join, since an entry in *Orders* can either have an associate value in **JourneyClasses_Id**, **RejseplanenProduct_Id** or None of them. To analyse *Orders* further, a few additional queries were performed. But the 'join' were done using the following query:
```SQL
SELECT COUNT(*)
FROM Orders
    JOIN SJSearchJourneys ON Orders.JourneyClasses_Id = SJSearchJourney.Id
WHERE Orders.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
--- ANSWER : **31,965**

We also make use of the Query:
```SQL
SELECT COUNT(*)
FROM Orders
WHERE Orders.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
--- ANSWER : **714,471**


## SJSearchJourneys and SJWaypoints

A *SearchJourney* have a one-to-many relationship with *Waypoints* 

```SQL
SELECT COUNT(*)
FROM SJSearchJourneys
    JOIN SJWaypoints ON SJSearchJourneys.Id = SJWaypoints.SJSearchJourney_Id
WHERE SJSearchJourneys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
--- ANSWER : **513,127**

```SQL
SELECT COUNT(*)
FROM SJSearchJourneys
WHERE SJSearchJourneys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
--- ANSWER : **31,965**

# Deep dive into a Extract from Journeys x Tickets where the journeys are from tickets baught a joint travel

The SQL command run:

```SQL
SELECT * 
FROM Tickets 
JOIN Journeys ON Tickets.Journey_Id = Journeys.Id
WHERE OrderId = 'ed70bcb8-bc89-4983-a068-0049dd5c249d'
```

The OrderId were found by running 

```SQL
SELECT OrderId, COUNT(*) AS OrderCount
FROM Tickets
GROUP BY OrderId
HAVING COUNT(*) > 2;
```

The OrderId was then chosen by selecting a relatively high number of OrderCount (36). Thus the first SQL query returns a table with 36 distinct journeys (based on journey id). 

Please take a look at Appendix_1.md for the full csv file. 

The data is highlighted in order to showcase the limits of our data. 
For the extracted data we can see that (of the relevant columns):
- The **Type** is set to *Zone*. This would be nice and awesome if the SJSearchJourneys actually contained detailed information on Journeys of type *Zone*, but this is not the case for all of these entries. 
- The **StartStop** and **EndStop** are for all journeys are left empty, not giving any information of the actual journey
- The **SearchStart** contain a value of 'Min lokation (01)' and thus does not give any indication of a location. 
- The **SearchEnd** contain a value of '2 zoner' gives just as much information.
- The **InternalValidZones** contain the value '1001,1002,1003' which tells us that the Ticket for the Journey is valid in these three zones. 
- The **InternalStartZone** likewise tells us that the journey possibly started in zone 1001. 'Possibly' since the column named **StartZone** is empty. 

Of course this is a, be it random, selected extracted table and thus further research have to be made in order to say whether the above observation is true for a magnitude of other 'joint' travels. But from the second query above, a (still counting) ~500.000 rows showcases the amount of joint travels, with most of them being around ~3-5 tickets but many with > 25 as well - probably institutes buying tickets for an entire classroom or similar. 

* Do note, that we also attempted to find these journeys in SJSearchJourneys but the order with the OrderId for the Tickets does not contain a value in JourneyClasses_Id thus indicating that there is no association between this journey and SJSearchJourneys. 

## Orders with no entry in SJ or RP

Looking into how many journeys in one month that do not, when joined with tickets and orders, have a JourneyClasses_Id as well as no RejseplanenProductId. These are found with the following query: 

```SQL
SELECT COUNT(*)
FROM Orders
WHERE Orders.JourneyClasses_Id is NULL and Orders.RejseplanenProduct_Id is NULL
and Orders.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```

The result is ***653373*** in one month. So we loose "only" 653373 journeys when joining on from Orders, so the majority of the lost journeys are not from here

# The JOINS
## HOW MANY JOURNEYS CAN WE ACTUALLY WORK WITH?


If we are to only work with data from the Journeys table, then we need to have either both SearchStart and SearchEnd or both StartStop and EndStop where these "pairs" do not have the same value. The number of journeys we can work with is found with the following query: 
```SQL
SELECT COUNT(*)
FROM Journeys
WHERE 
    (Journeys.SearchStart IS NOT NULL AND Journeys.SearchEnd IS NOT NULL AND Journeys.SearchStart <> Journeys.SearchEnd)
    OR
    (Journeys.StartStop IS NOT NULL AND Journeys.EndStop IS NOT NULL AND Journeys.StartStop <> Journeys.EndStop);
```

The result is that from the entire Journeys table which contains around 43 million journeys, we have ***23449428*** we can "work" with. 

``` SQL
SELECT COUNT(*)
FROM Journeys
JOIN Tickets ON Journeys.Id = Tickets.Journey_Id
WHERE Journeys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```

***870115***

``` SQL
SELECT COUNT(*)
FROM Journeys
JOIN Tickets ON Journeys.Id = Tickets.Journey_Id
JOIN Orders ON Orders.Id = Tickets.OrderId
WHERE Journeys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
***870115***
This means that for each ticket there exists an order. So the one to many relationship between tickets and orders is not clear in this month at least. 

##### SJ

``` SQL
SELECT COUNT(*)
FROM Journeys
JOIN Tickets ON Journeys.Id = Tickets.Journey_Id
JOIN Orders ON Orders.Id = Tickets.OrderId
JOIN SJSearchJourneys ON Orders.JourneyClasses_Id = SJSearchJourneys.Id
WHERE Journeys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
***35948***
Joining with SJ we get 35948 journeys for a month. So we loose around 834167 journeys in this join

``` SQL
SELECT COUNT(*)
FROM Journeys
JOIN Tickets ON Journeys.Id = Tickets.Journey_Id
JOIN Orders ON Orders.Id = Tickets.OrderId
JOIN SJSearchJourneys ON Orders.JourneyClasses_Id = SJSearchJourneys.Id
JOIN SJWaypoints ON SJSearchJourneys.Id =  SJWaypoints.SJSearchJourney_Id
WHERE Journeys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
***571087***
The number of waypoints for a single month

##### RP

``` SQL
SELECT COUNT(*)
FROM Journeys
JOIN Tickets ON Journeys.Id = Tickets.Journey_Id
JOIN Orders ON Orders.Id = Tickets.OrderId
JOIN RejseplanenProducts ON Orders.RejseplanenProduct_Id = RejseplanenProducts.Id
WHERE Journeys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
***49350***
Joining with RP we get 49350 which is more journeys than with SJ. This could indicate that the "major" part of the DOT-billet app business is indeed the "zone-kort", "pendlerkort" and cards alike. 

``` SQL
SELECT COUNT(*)
FROM Journeys
JOIN Tickets ON Journeys.Id = Tickets.Journey_Id
JOIN Orders ON Orders.Id = Tickets.OrderId
JOIN RejseplanenProducts ON Orders.RejseplanenProduct_Id = RejseplanenProducts.Id
JOIN RPWaypoints ON RejseplanenProducts.Id = RPWaypoints.RejseplanenProduct_Id
WHERE Journeys.CreatedOn BETWEEN '2022/12/01 00:00:00' and '2023/01/01 00:00:00'
```
***394597***
The amount of waypoints (RP here) is smaller than the total number when joining with RP. This could make sense because of the different cards leaving no information about what journeys people take. 

# Check Journeys for RP entries



In [10]:
import pandas as pd

In [11]:
df_journeys = pd.read_csv('cph_file.csv')

In [9]:
df_orders = pd.read_csv('Orders.csv', usecols=['Id', 'RejseplanenProduct_Id'])


In [10]:
df_tickets = pd.read_csv('Tickets.csv', usecols=['OrderId', 'Id', 'Journey_Id'])

In [11]:
df_orders_RP = df_orders[(df_orders['RejseplanenProduct_Id'].notna())]
df_orders_RP




,Id,RejseplanenProduct_Id
62,0311120d-ff56-413a-957f-000018cb9d50,611a2832-f1d8-46b8-b3b1-f8e5897ffd11
67,919f1e1c-cd9f-437b-93a4-00001b8a4349,8e6e14fe-c657-4a9f-8722-155f91d3556e
72,5a52fe8a-fc8b-4549-b76e-00001c80b6af,e42800a7-681e-4820-85df-af90a22c12ee
82,f0afd355-48a4-4bfb-8374-0000207f743e,6a6c7671-7247-4a00-8250-8560091aa93a
83,872690ee-5ba0-40a1-8512-000020c72dc5,b8f4d336-f2bd-4797-ad57-939a0f69db16
...,...,...
33062058,94eb9c44-e991-423a-992b-ffffda8047d0,d9e045d5-5f46-4524-8834-badbab177c05
33062072,b2c610f7-7aeb-41f3-9754-ffffe1d77eff,c30961bf-cc11-4fde-ae24-9382cca02d14
33062077,5b49b0d5-0be1-4a4e-984f-ffffe57ec021,c65f0159-da33-414a-bdd9-7c8ff224c3b3
33062098,d6851721-e037-45e8-b93d-fffff1630028,f41b2ea6-4f9b-470d-9474-6199a37beb1d


In [7]:
matching = pd.merge(df_tickets, df_orders_RP, left_on='OrderId', right_on='Id')

In [8]:
matching

,Id_x,OrderId,Journey_Id,Id_y,RejseplanenProduct_Id
0,107b976c-91b6-486c-a6fc-000002f6b282,b50ff118-f2a8-4f01-bc24-cf34c3de53aa,6f50f4af-f3a5-4c91-a5f9-d7eefcebd361,b50ff118-f2a8-4f01-bc24-cf34c3de53aa,9070944d-aaa6-45f3-bbbf-2561dfce9812
1,260a920e-be8f-4e81-b389-00000ea17ed4,c27cef38-1193-4e85-834a-0a0c3f82f036,5cdecc86-6408-49bb-b984-4f2554ce7d28,c27cef38-1193-4e85-834a-0a0c3f82f036,fff03199-9f09-427b-bfc8-26be2dd6c1a4
2,7ee73af6-093f-4692-a5fd-000020b17790,e13256af-d766-4db6-825f-ce961dd66c20,de615f41-9131-4088-b2bc-dbbf428a116e,e13256af-d766-4db6-825f-ce961dd66c20,e69caab4-326e-4254-ae3f-1a0676490d4c
3,75a01b2b-b26c-40f0-8102-000025652193,c1884911-3600-4023-aa84-15293cccbfde,e2450d76-44a7-4eba-af04-7101c9342326,c1884911-3600-4023-aa84-15293cccbfde,54fd682e-6a9c-4c64-a1d1-213956410460
4,7feb5bea-bda9-4cb1-8b7a-00002856d425,163431ba-35ab-49e8-872e-c424f63f4de0,7c506fe5-8c49-435d-b9fb-caacf62667dd,163431ba-35ab-49e8-872e-c424f63f4de0,f85423d0-99f9-42b4-a8b6-7e82afdf1eda
...,...,...,...,...,...
2430339,c05174fb-89f0-468f-9265-fffff14f2b04,02cb70fc-0830-4066-b84e-9067c9e30501,cfdd2cfa-585a-4a05-8b29-2f8842b850f6,02cb70fc-0830-4066-b84e-9067c9e30501,8fb33989-ddc8-4f83-96e4-15a3f5f272c1
2430340,60e3e8a3-617c-41b9-8db4-fffff9521688,b480aabf-ca93-4b14-b86e-00576714edfb,71e63f80-e3a0-4e82-96f4-3b83c1d733ee,b480aabf-ca93-4b14-b86e-00576714edfb,bba770ac-a1d3-4f25-b322-6b2d24a94893
2430341,60292d47-3e30-49f5-b7ae-fffffa0bb851,9aa4d40b-ede4-4b86-82f6-18aa573a94ba,363c2a31-52e9-42bf-af6c-6c49790e99eb,9aa4d40b-ede4-4b86-82f6-18aa573a94ba,d9051100-a787-498e-82a6-c5466eb8e8d3
2430342,d692b971-bbdc-4ccb-836e-fffffb6b8a1f,a04a865f-dc39-4a8a-b96a-b16b602b000a,410e4510-98c5-4bbe-bc6b-f1a1c1daa122,a04a865f-dc39-4a8a-b96a-b16b602b000a,d5215807-f6f5-484e-87e0-68d7d5897da6


In [9]:
journeys_with_rp = pd.merge(df_journeys, matching, left_on='Id', right_on='Journey_Id')
journeys_with_rp

,level_0,index,Id,AmountOfZones,EndZone,SearchStart,SearchEnd,ModifiedOn,CreatedOn,Id_x,OrderId,Journey_Id,Id_y,RejseplanenProduct_Id
0,37386341,37386341,bc31ae04-e2d7-4c21-ae7e-dccc40f42ec4,4,0,"Borups Allé 270, 2400 København NV, Københavns...",CPH Lufthavn,2020-08-23 14.09.45.4138258,2020-08-23 14.09.45.4138258,02225c31-4583-411a-a720-3c7eef69b199,4c00aa4b-de9b-4405-b51e-67b3ef42e518,bc31ae04-e2d7-4c21-ae7e-dccc40f42ec4,4c00aa4b-de9b-4405-b51e-67b3ef42e518,ce6c934e-5070-45a4-b4d7-7452af041433
1,42055157,42055157,cc90a133-1f4e-4d0b-ae3a-f861425b2191,4,0,"Borups Allé 270, 2400 København NV, Københavns...",CPH Lufthavn,2020-08-23 14.09.45.4138258,2020-08-23 14.09.45.4138258,cfc5c365-114a-45b4-8439-f02478cca684,4c00aa4b-de9b-4405-b51e-67b3ef42e518,cc90a133-1f4e-4d0b-ae3a-f861425b2191,4c00aa4b-de9b-4405-b51e-67b3ef42e518,ce6c934e-5070-45a4-b4d7-7452af041433


# Journeys Count of null in SearchStart vs StartStop

In [2]:
import pandas as pd
df = pd.read_csv('../../Data/All_Journeys.csv')

In [3]:
count_notnull_searchStart = df[(pd.notna(df['SearchStart']))].count()
count_notnull_StartStop = df[(pd.notna(df['StartStop']))].count()

In [6]:
count_notnull_searchStart['Id']

25758204

In [7]:
count_notnull_StartStop['Id']

665526

In [8]:
df.count()

Id                    43345946
Type                  31791670
internalStartZones    32124423
StartZone             43345946
internalValidZones    43343110
StartStop               665526
AmountOfZones         43345946
EndZone               43345946
EndStop                 665526
SearchStart           25758204
SearchEnd             22315171
ModifiedOn            43345946
CreatedOn             43345946
JourneyClasses_Id            0
TravelType             5563619
ExtraFrom                    0
ExtraTo                      0
dtype: int64

In [12]:
matching = pd.merge(df_tickets, df_orders_RP, left_on='OrderId', right_on='Id')
journeys_with_rp = pd.merge(df, matching, left_on='Id', right_on='Journey_Id')

In [13]:
journeys_with_rp

,Id,Type,internalStartZones,StartZone,internalValidZones,StartStop,AmountOfZones,EndZone,EndStop,SearchStart,...,CreatedOn,JourneyClasses_Id,TravelType,ExtraFrom,ExtraTo,Id_x,OrderId,Journey_Id,Id_y,RejseplanenProduct_Id
0,cbd5ad3b-0bf0-4314-bd74-000001a41c82,NaN,NaN,0,"1001,1002,1003",NaN,3,0,NaN,Femøren St. (Metro),...,2023-08-04 08.33.59.0415651,NaN,NaN,NaN,NaN,b9f9e2b8-72c8-4a39-8801-7047bc599034,b4758852-deae-43e3-a5cc-2d4d336a29b3,cbd5ad3b-0bf0-4314-bd74-000001a41c82,b4758852-deae-43e3-a5cc-2d4d336a29b3,dd8200dd-dc7a-44a0-bec6-a21a6043bb08
1,578ae600-b1a9-4c65-b5bd-00000e725dee,NaN,NaN,0,"1002,1001,1003",NaN,3,0,NaN,NaN,...,2023-01-31 06.08.56.4705545,NaN,NaN,NaN,NaN,ac02d066-4cbd-4a2f-b65e-cc6163eedb12,43ac8b96-e8a6-4f23-a120-f7a9ccd9fa7f,578ae600-b1a9-4c65-b5bd-00000e725dee,43ac8b96-e8a6-4f23-a120-f7a9ccd9fa7f,9f1455eb-29d4-444c-9bef-35415c01584f
2,7615a1a3-2049-465b-95db-0000111bb081,NaN,1009,0,"1001,1002,1009,1030,1031,1040,1041,1042,1050,1...",NaN,9,1001,NaN,NaN,...,2020-09-02 08.25.36.7253141,NaN,NaN,NaN,NaN,0fc35109-3753-4075-bd88-d524e1ccf429,1de0d89a-e473-4c23-b2f5-afc5e2b087a6,7615a1a3-2049-465b-95db-0000111bb081,1de0d89a-e473-4c23-b2f5-afc5e2b087a6,21d53113-d848-4fde-a9ca-b2fee9c1f13c
3,32c6d9f6-7ece-4795-9c90-000014d2a70c,NaN,NaN,0,"1001,1002,1003",NaN,3,0,NaN,NaN,...,2023-09-25 07.32.24.7382288,NaN,NaN,NaN,NaN,186c45e7-c01f-4399-8a01-065ed8ff8dfd,a9f6f6ea-8f1f-4257-b08b-90b41a417d69,32c6d9f6-7ece-4795-9c90-000014d2a70c,a9f6f6ea-8f1f-4257-b08b-90b41a417d69,1e97efc9-c20c-4b58-85de-7a326bc183d4
4,76f8d512-5f05-41e5-b917-00001703de10,NaN,NaN,0,"1001,1002",NaN,2,0,NaN,NaN,...,2023-01-23 16.56.35.4395595,NaN,NaN,NaN,NaN,c45462df-9c99-44e8-a9f7-428c7e5856fe,a8c26d23-21e1-45b4-868e-56ab51117334,76f8d512-5f05-41e5-b917-00001703de10,a8c26d23-21e1-45b4-868e-56ab51117334,fa95822c-f0ee-4f2e-9b6f-7b044eb95c9c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2360981,92cfbbba-ced9-45a3-88c2-ffffd2a77030,NaN,NaN,0,"1001,1003",NaN,2,0,NaN,Bella Center St. (Metro),...,2022-12-16 13.51.54.2439066,NaN,NaN,NaN,NaN,c65ce9cf-7995-4981-961a-891df16db38a,246fcf10-b3a0-417f-81ba-fb4095a2012d,92cfbbba-ced9-45a3-88c2-ffffd2a77030,246fcf10-b3a0-417f-81ba-fb4095a2012d,8082f07e-c6e1-4f44-ac93-81d402d29077
2360982,1ba01277-c32f-4c01-80fe-ffffd895ff77,NaN,NaN,0,"1001,1002",NaN,2,0,NaN,NaN,...,2022-06-02 05.54.36.9914136,NaN,NaN,NaN,NaN,b9981816-4ade-479e-ad40-bef42056fef6,17727a1f-104b-4547-b35f-485b455af65b,1ba01277-c32f-4c01-80fe-ffffd895ff77,17727a1f-104b-4547-b35f-485b455af65b,5c34a597-2ee0-4172-9e34-368c83c0cc09
2360983,7a962e52-77f1-4878-b3b3-ffffddac3957,NaN,NaN,0,"1001,1002,1030,1041,1051",NaN,5,0,NaN,Knippelsbro (Torvegade),...,2023-10-04 19.30.31.1786990,NaN,NaN,NaN,NaN,d9575b6e-ba2a-4cd8-a098-2e12963317ae,0d025478-953d-4ce8-b072-f25bc6ccdfcc,7a962e52-77f1-4878-b3b3-ffffddac3957,0d025478-953d-4ce8-b072-f25bc6ccdfcc,af6e60de-2d19-4d92-98c5-41fb0aeba13a
2360984,80e71052-9245-40ec-9f83-fffffa1ca8f4,NaN,NaN,0,"1001,1002",NaN,2,0,NaN,My Location (02),...,2021-04-25 14.35.25.8673301,NaN,NaN,NaN,NaN,80d5fdd4-837c-4f77-9bb0-58785d40e610,e4dc5d5e-58ba-478b-82ed-03a2de6fd87c,80e71052-9245-40ec-9f83-fffffa1ca8f4,e4dc5d5e-58ba-478b-82ed-03a2de6fd87c,54f3ed2c-e242-4129-aaf9-67611bf73101
